<a href="https://colab.research.google.com/github/IG-Reagan/Mathematical-Modelling_Network-Flow-Optimisation_Paint-Transportation-Problem/blob/main/paint_transport_LP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paint Transportation Optimisation (Linear Programming Model)
**Author:** Giwa Iziomo  
**Project:** Mathematical Modelling - Network Flow Optimisation (Transportation Problem)

This notebook determines the minimum-cost weekly transport schedule for a UK paint manufacturer. Two factories supply paint either directly to five wholesalers or through three warehouses.

**Model Type:** Linear Programming (LP) using PuLP.

In [10]:
# Install PuLP

!pip install pulp

In [11]:
# Import PuLP

import pulp

In [5]:
# -----------------------------
# 1. Sets and Data
# -----------------------------

suppliers = ["Bristol", "Leeds", "London", "Birmingham", "Glasgow"]
destinations = [
    "London", "Birmingham", "Glasgow",
    "Wholesaler 1", "Wholesaler 2", "Wholesaler 3", "Wholesaler 4", "Wholesaler 5"
]

# Transportation cost (£ per ton)
# Infeasible routes assigned a prohibitive cost of £500/t
cost = {
    ("Bristol", "London"): 25, ("Bristol", "Birmingham"): 23,
    ("Bristol", "Glasgow"): 500, ("Bristol", "Wholesaler 1"): 80,
    ("Bristol", "Wholesaler 2"): 500, ("Bristol", "Wholesaler 3"): 90,
    ("Bristol", "Wholesaler 4"): 100, ("Bristol", "Wholesaler 5"): 86,

    ("Leeds", "London"): 30, ("Leeds", "Birmingham"): 27, ("Leeds", "Glasgow"): 30,
    ("Leeds", "Wholesaler 1"): 500, ("Leeds", "Wholesaler 2"): 70,
    ("Leeds", "Wholesaler 3"): 54, ("Leeds", "Wholesaler 4"): 500,
    ("Leeds", "Wholesaler 5"): 100,

    ("London", "London"): 500, ("London", "Birmingham"): 500, ("London", "Glasgow"): 500,
    ("London", "Wholesaler 1"): 37, ("London", "Wholesaler 2"): 31,
    ("London", "Wholesaler 3"): 500, ("London", "Wholesaler 4"): 40,
    ("London", "Wholesaler 5"): 44,

    ("Birmingham", "London"): 500, ("Birmingham", "Birmingham"): 500, ("Birmingham", "Glasgow"): 500,
    ("Birmingham", "Wholesaler 1"): 36, ("Birmingham", "Wholesaler 2"): 40,
    ("Birmingham", "Wholesaler 3"): 43, ("Birmingham", "Wholesaler 4"): 40,
    ("Birmingham", "Wholesaler 5"): 46,

    ("Glasgow", "London"): 500, ("Glasgow", "Birmingham"): 500, ("Glasgow", "Glasgow"): 500,
    ("Glasgow", "Wholesaler 1"): 45, ("Glasgow", "Wholesaler 2"): 42,
    ("Glasgow", "Wholesaler 3"): 30, ("Glasgow", "Wholesaler 4"): 500,
    ("Glasgow", "Wholesaler 5"): 36,
}

# Supply limits (tons)
supply = {"Bristol": 40000, "Leeds": 50000, "London": 20000,
          "Birmingham": 15000, "Glasgow": 12000}

# Demand requirements (tons)
demand = {
    "London": 20000, "Birmingham": 15000, "Glasgow": 12000,
    "Wholesaler 1": 15000, "Wholesaler 2": 20000, "Wholesaler 3": 13000,
    "Wholesaler 4": 14000, "Wholesaler 5": 16000
}

In [6]:
# -----------------------------
# 2. Model Definition
# -----------------------------

model = pulp.LpProblem("Paint_Transportation_3_Warehouses", pulp.LpMinimize)

# Decision variables: quantity shipped from supplier i to destination j (tons)
x = pulp.LpVariable.dicts("ship", (suppliers, destinations),
                          lowBound=0, cat="Continuous")

# Objective function: minimise total weekly transport cost
model += pulp.lpSum(cost[(i, j)] * x[i][j]
                    for i in suppliers for j in destinations if (i, j) in cost)

In [7]:
# -----------------------------
# 3. Constraints
# -----------------------------

# Supply constraints
for i in suppliers:
    model += pulp.lpSum(x[i][j] for j in destinations if (i, j) in cost) <= supply[i], f"Supply_{i}"

# Demand constraints
for j in destinations:
    model += pulp.lpSum(x[i][j] for i in suppliers if (i, j) in cost) >= demand[j], f"Demand_{j}"


In [8]:
# -----------------------------
# 4. Solve
# -----------------------------

model.solve(pulp.PULP_CBC_CMD(msg=False))

1

In [9]:
# -----------------------------
# 5. Results
# -----------------------------

print("Status:", pulp.LpStatus[model.status])
print(f"Minimum total weekly transport cost: £{pulp.value(model.objective):,.0f}\n")

print("Optimal Shipment Plan (only non-zero routes):")
print("-" * 65)
for i in suppliers:
    for j in destinations:
        if (i, j) in cost and x[i][j].value() > 1e-6:
            tons = x[i][j].value()
            route_cost = cost[(i, j)]
            print(f"{i:<10} → {j:<12} : {tons:8,.0f} tons  @ £{route_cost}/t   (£{tons*route_cost:,.0f})")

Status: Optimal
Minimum total weekly transport cost: £4,937,000

Optimal Shipment Plan (only non-zero routes):
-----------------------------------------------------------------
Bristol    → London       :   20,000 tons  @ £25/t   (£500,000)
Bristol    → Birmingham   :   15,000 tons  @ £23/t   (£345,000)
Leeds      → Glasgow      :   12,000 tons  @ £30/t   (£360,000)
Leeds      → Wholesaler 2 :   18,000 tons  @ £70/t   (£1,260,000)
Leeds      → Wholesaler 3 :   13,000 tons  @ £54/t   (£702,000)
London     → Wholesaler 2 :    2,000 tons  @ £31/t   (£62,000)
London     → Wholesaler 4 :   14,000 tons  @ £40/t   (£560,000)
London     → Wholesaler 5 :    4,000 tons  @ £44/t   (£176,000)
Birmingham → Wholesaler 1 :   15,000 tons  @ £36/t   (£540,000)
Glasgow    → Wholesaler 5 :   12,000 tons  @ £36/t   (£432,000)
